# GAM Coach: Binary Classification

In this notebook, we will learn how to generate *diverse* and *customizable* counterfactual explanations for Generalized Additive Models (GAMs).

## 2. Adult


In [1]:
%load_ext autoreload
%autoreload 2

import gamcoach as coach

import numpy as np
import pandas as pd

import dice_ml
import urllib.request
import json
import pickle

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter
from tqdm import tqdm

SEED = 101221

In [2]:
adult_data = pd.read_csv('./data/adult.data', header=None)
adult_data_test = pd.read_csv('./data/adult.test', header=None)

# From https://github.com/amirhk/mace/blob/01e6a405ff74e24dc3438a005cd60892154d189d/_data_main/fair_adult_data.py
adult_attrs = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
]

selected_features = [0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13]

x_train = adult_data.to_numpy()[:, selected_features]

y_train_ints = list(map(lambda x: 0 if x == ' <=50K' else 1, adult_data.iloc[:, -1].tolist()))
y_train = np.array(y_train_ints)

x_test = adult_data_test.to_numpy()[:, selected_features]

y_test_ints = list(map(lambda x: 0 if x == ' <=50K.' else 1, adult_data_test.iloc[:, -1].tolist()))
y_test = np.array(y_test_ints)

adult_feature_names = np.array(adult_attrs)[selected_features]
adult_feature_names
adult_cont_indexes = [0, 3, 8, 9, 10]
adult_feature_types = [
    "continuous" if i in adult_cont_indexes else "categorical"
    for i in range(len(adult_feature_names))
]

In [3]:
# Train an EBM classifier
adult_ebm = ExplainableBoostingClassifier(
    feature_names=adult_feature_names,
    feature_types=adult_feature_types,
    random_state=SEED,
)
adult_ebm.fit(x_train, y_train)


ExplainableBoostingClassifier(feature_names=['age', 'workclass', 'education',
                                             'education_num', 'marital_status',
                                             'occupation', 'relationship',
                                             'sex', 'capital_gain',
                                             'capital_loss', 'hours_per_week',
                                             'native_country',
                                             'relationship x hours_per_week',
                                             'age x relationship',
                                             'age x capital_loss',
                                             'marital_status x hours_per_week',
                                             'relationship x capital_loss',
                                             'education_num x o...
                                             'workclass x relationship'],
                              feature_types=['co

In [7]:
# Evaluate our model

y_pred = adult_ebm.predict(x_test)
y_pred_prob = adult_ebm.predict_proba(x_test)[:, 1]

print(Counter(y_pred))
print()

accuracy = metrics.accuracy_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred_prob)
f1 = metrics.f1_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

tn = confusion_matrix[0, 0]
fn = confusion_matrix[1, 0]
fp = confusion_matrix[0, 1]
tp = confusion_matrix[1, 1]
specificity = tn / (tn + fp)

temp = ('accuracy: {:.4f} \nauc: {:.4f} \nrecall:{:.4f} \nprecision: {:.4f} '+
    'specificity: {:.4f} \nf1: {:.4f}\nbalanced accuracy: {:.4f}')
print(temp.format(accuracy, auc, recall, precision, specificity, f1, balanced_accuracy))
print()

print('confusion matrix:\n', confusion_matrix)

Counter({0: 13103, 1: 3178})

accuracy: 0.8727 
auc: 0.9256 
recall:0.6438 
precision: 0.7791 specificity: 0.9435 
f1: 0.7050
balanced accuracy: 0.7937

confusion matrix:
 [[11733   702]
 [ 1370  2476]]


In [8]:
# Find an interesting data point
# We can focus on test cases where our model rejcets the application (y_hat = 0)
reject_index = y_pred == 0
x_reject = x_test[reject_index, :]
y_pred_reject = y_pred[reject_index]

explain_df = pd.DataFrame(x_reject).sample(400)
explain_df.columns = adult_feature_names

reject_df = pd.DataFrame(np.hstack((x_reject, y_pred_reject.reshape(-1, 1))))
reject_df.columns = adult_feature_names.tolist() + ['prediction']
reject_df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,prediction
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Male,0,0,40,United-States,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,50,United-States,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,Male,0,0,40,United-States,0
3,18,?,Some-college,10,Never-married,?,Own-child,Female,0,0,30,United-States,0
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,Male,0,0,30,United-States,0


In [14]:
def evaluate_cf(x1, x2, feature_names, cont_mads):
    cur_distance = 0
    feature_weight = 1 / len(feature_names)
    changed_feature = 0

    for i in range(len(feature_names)):
        cur_name = feature_names[i]

        # If the current feature is continuous
        if cur_name in cont_mads:
            if cont_mads[cur_name] > 0:
                cur_distance += (
                    feature_weight * abs(float(x2[i]) - float(x1[i])) / cont_mads[cur_name]
                )
            else:
                cur_distance += (
                    feature_weight * abs(float(x2[i]) - float(x1[i]))
                )
        else:
            cur_distance += 1 if x1[i] != x2[i] else 0

        if x1[i] != x2[i]:
            changed_feature += 1

    return cur_distance, changed_feature


### 1.1. GAM Coach

In [10]:
features_to_vary = adult_feature_names
adult_feature_names

array(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country'], dtype='<U14')

In [11]:
my_coach = coach.GAMCoach(adult_ebm, x_train, adjust_cat_distance=False)

cfs = []
for i in tqdm(range(explain_df.shape[0])):
# for i in tqdm(range(20)):

    try:
        cf = my_coach.generate_cfs(
            explain_df.iloc[i, :].to_numpy(),
            total_cfs=1,
            verbose=0,
            categorical_weight=1
            # Some continuous features need to have integer values in practice
            # continuous_integer_features=["open_acc", "total_acc", "mort_acc", "fico_score"],
        )

        cfs.append([explain_df.iloc[i, :], cf.to_df().iloc[0, :]])
    except:
        cfs.append(None)

100%|██████████| 400/400 [07:52<00:00,  1.18s/it]


In [12]:
pickle.dump(cfs, open('./data/adult-coach-cfs.pkl', 'wb'))

In [15]:
distances, feature_nums, failed_indexes = [], [], []

for i in range(len(cfs)):
    if cfs[i] is None:
        failed_indexes.append(i)
        continue

    x1 = cfs[i][0]
    x2 = cfs[i][1][:-1]

    cur_distance, changed_feature = evaluate_cf(x1, x2, adult_feature_names, my_coach.cont_mads)
    
    distances.append(cur_distance)
    feature_nums.append(changed_feature)

In [16]:
print(
    "coach",
    "\ndistance: ",
    np.mean(distances),
    "\nnum: ",
    np.mean(feature_nums),
    "\nfails: ",
    len(failed_indexes),
)


coach 
distance:  1.6855904527777779 
num:  2.4075 
fails:  0


### 1.2. Genetic Algorithm

In [19]:
dice_df = pd.DataFrame(x_train)
dice_df.columns = adult_feature_names
dice_df['status'] = y_train

feature_names_cont = adult_feature_names[adult_cont_indexes].tolist()

for c in feature_names_cont:
    dice_df[c] = dice_df[c].astype(float)

dice_data = dice_ml.Data(
    dataframe=dice_df,
    continuous_features=feature_names_cont,
    outcome_name='status'
)

dice_model = dice_ml.Model(model=adult_ebm, backend='sklearn')

In [21]:
exp = dice_ml.Dice(dice_data, dice_model, method='genetic')

dice_rejected_df = explain_df.copy()
for c in feature_names_cont:
    dice_rejected_df[c] = dice_rejected_df[c].astype(float)

explanation_generic = exp.generate_counterfactuals(
    dice_rejected_df,
    total_CFs=1,
    features_to_vary=features_to_vary.tolist(),
    # posthoc_sparsity_param=0.2,
    verbose=False,
    desired_class="opposite")

100%|██████████| 400/400 [04:43<00:00,  1.41it/s]


In [22]:
g_distances, g_feature_nums, g_failed_indexes = [], [], []

for i in range(len(explanation_generic.cf_examples_list)):
    if explanation_generic.cf_examples_list[i].final_cfs_df is None:
        g_failed_indexes.append(i)
        continue

    x1 = dice_rejected_df.iloc[i, :]
    x2 = explanation_generic.cf_examples_list[i].final_cfs_df.iloc[0, :]

    cur_distance, changed_feature = evaluate_cf(x1, x2, adult_feature_names, my_coach.cont_mads)
    
    g_distances.append(cur_distance)
    g_feature_nums.append(changed_feature)


In [23]:
print(
    "generic",
    "\ndistance: ",
    np.mean(g_distances),
    "\nnum: ",
    np.mean(g_feature_nums),
    "\nfails: ",
    len(g_failed_indexes),
)


generic 
distance:  4.923069444444444 
num:  4.6475 
fails:  0


### KD Tree

In [25]:
exp = dice_ml.Dice(dice_data, dice_model, method='kdtree')

dice_rejected_df = explain_df.copy()
for c in feature_names_cont:
    dice_rejected_df[c] = dice_rejected_df[c].astype(float)

explanation_tree = exp.generate_counterfactuals(
    dice_rejected_df,
    total_CFs=1,
    features_to_vary=features_to_vary.tolist(),
    # posthoc_sparsity_param=0.2,
    verbose=False,
    desired_class="opposite")

100%|██████████| 400/400 [04:41<00:00,  1.42it/s]


In [26]:
k_distances, k_feature_nums, k_failed_indexes = [], [], []

for i in range(len(explanation_tree.cf_examples_list)):
    if explanation_tree.cf_examples_list[i].final_cfs_df is None:
        k_failed_indexes.append(i)
        continue

    if explanation_tree.cf_examples_list[i].final_cfs_df.shape == (0, 0):
        k_failed_indexes.append(i)
        continue

    x1 = dice_rejected_df.iloc[i, :]
    x2 = explanation_tree.cf_examples_list[i].final_cfs_df.iloc[0, :]

    cur_distance, changed_feature = evaluate_cf(x1, x2, adult_feature_names, my_coach.cont_mads)
    
    k_distances.append(cur_distance)
    k_feature_nums.append(changed_feature)


In [27]:
print(
    "tree",
    "\ndistance: ",
    np.mean(k_distances),
    "\nnum: ",
    np.mean(k_feature_nums),
    "\nfails: ",
    len(k_failed_indexes),
)


tree 
distance:  5.108152777777778 
num:  4.95 
fails:  0
